In [1]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model
     

c:\Siddhartha\redbrick-hacks-2023-submission-unstable\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [2]:

import os
import pinecone

# get api key from app.pinecone.io
api_key = "d9b7dbe2-d68d-4381-937d-06ddb5cd761c"
# find your environment next to the api key in pinecone console
env = "asia-southeast1-gcp-free"

pinecone.init(
    api_key=api_key,
    environment=env
)

In [3]:
index_name = 'semantic-search'
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine'
    )

# now connect to the index
index = pinecone.GRPCIndex(index_name)

In [8]:
import csv
to_upsert = []

with open("file1.csv", 'r',encoding='utf-8') as csvfile:
        data = csv.reader(csvfile)
        for lines in data:
            if(lines == [] or lines[0] == "id"):
                continue
            if (lines[1] != ""):
                to_upsert.append([lines[0],lines[5]])
print(to_upsert)
questions = []
ids = []
for k in range(len(to_upsert)):
    ids.append(str(to_upsert[k][0]))
    questions.append(to_upsert[k][1])

ids

[['1', "The true story of physicist J. Robert Oppenheimer's journey from driven and ambitious scientist to remorseful and tormented man struck hard by the gravity of what he had done. For Oppenheimer, the successful detonation of the first atom bomb validates years of tireless work. But in this moment of seeming triumph, Oppenheimer sees before him a destructive power of almost supernatural magnitude."], ['2', "Over 133 years in the making, from humble beginnings manufacturing 'Hanufuda' cards came one of the world's most recognized videogame companies, from the birth of Mario and Luigi to Donkey Kong and Zelda... to beating its competition and presenting itself as a platform for quality games and strong values. This is the story of Nintendo."], ['3', "Immerse yourself in the enigmatic realm of global cybersecurity with this riveting documentary that navigates the operations of the NSO Group. As a private entity leading the charge in cyber intelligence technologies, Group's flagship sp

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '13

In [9]:
from tqdm.auto import tqdm

batch_size = 128

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

# check number of records in the index
index.describe_index_stats()

100%|██████████| 7/7 [01:06<00:00,  9.50s/it]


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 128}},
 'total_vector_count': 128}